# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [17]:
# %%timeit -n 1 -r 1
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst so that they can use statistical modeling and machine learning to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: 
{
"coworker": "Data Analyst",
"task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning to identify trends and predict market movements",
"context": "The task involves using advanced data analysis techniques to track market data for AAPL, apply statistical modeling and machine learning algorithms to identify trends, and predict future market movements. The goal is to provide insights and alerts

Final Answer: Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science 2024 election Originals The 360 Newsletters Life Health COVID-19 Fall allergies Health news Mental health Relax Sexual health Studies The Unwind Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Buying guides Food Travel Autos Gift ideas Buying guides Entertainment Celebrity TV Movies Music How to Watch Interviews Videos Shopping Finance My Portfolio News Latest News Stock Market Originals Premium News Economics Housing Earnings Tech Crypto Biden Economy Markets Stocks: Most Actives Stocks: Gainers Stocks: Losers Trending Tickers Futures World Indices US Treasury Bonds Rates Currencies Crypto Top ETFs Top Mutual Funds Options: Highest Open Interest Options: Highest Implied Volatility Sectors Basic Materials Commu

Thought: I need to ask the Data Analyst a question to ensure they have all the information they need to successfully monitor and analyze market data for AAPL.

Action: Ask question to co-worker
Action Input: 
{
"coworker": "Data Analyst",
"question": "What tools and software will you use to monitor and analyze market data for AAPL?",
"context": "The task involves continuously monitoring and analyzing market data for AAPL using statistical modeling and machine learning to identify trends and predict market movements. It is essential to have the right tools and software for accurate analysis."
}


> Entering new CrewAgentExecutor chain...
I need to find the tools and software that are specifically focused on monitoring and analyzing market data for AAPL. I should use the tools provided to gather this information accurately.

Action: Read website content
Action Input: {"website_url": "https://www.nasdaq.com/market-activity/stocks/aapl"} 

I encountered an error while trying to use the too

I need to ensure that the Trading Strategy Developer creates potential trading strategies for AAPL that align with the user's risk tolerance.

Action: Delegate work to co-worker
Action Input: {"coworker": "Trading Strategy Developer", "task": "Develop potential trading strategies for AAPL based on user's risk tolerance and trading preferences (Day Trading)", "context": "The user's risk tolerance is medium, and they prefer Day Trading as their trading style. The Data Analyst is continuously monitoring and analyzing market data for AAPL to provide insights and alerts about significant market opportunities or threats."}

> Entering new CrewAgentExecutor chain...
I need to gather information on potential trading strategies for AAPL that align with the user's risk tolerance and day trading preferences. I should utilize the insights and alerts provided by the Data Analyst Agent to inform my strategy development.

Action: Search the internet
Action Input: {"search_query": "Day trading strateg

Final Answer: 
After reading the content from the provided link on three potential trading strategies for AAPL, I have gathered detailed information on each strategy:

1. Two-day high/low trading:
This strategy involves buying the break from the high of the two days or selling at the break from a two-day low. It is ideal for short-term traders and requires planning before the market opens. Risk management is essential, and traders should place a stop at the low of the previous session.

2. Low volatility ranges:
During periods of low volatility, traders can benefit from monitoring price action on short time frames using an oscillator. This method helps identify overbought or oversold levels and allows for short or long positions based on price fluctuations.

3. Gap fill:
This strategy focuses on price gaps from the previous trading day session, with common gaps often filled during the same day's session. Traders can plan for this strategy by marking the gap and placing long or short po

Final Answer: 
Based on the detailed analysis of the three day trading strategies for Apple stocks provided on the AskTraders website, here are the optimal execution plans for each strategy:

1. Two-day high/low trading:
- Identify the high and low from the past two trading days.
- Switch to a five-minute session and wait for a close outside of these points.
- Initiate a position based on the break from the high or low.
- Place a stop at the low of the previous session for risk management.
- Partially close the position if possible and adjust the remaining position to break even.

2. Low volatility ranges:
- Monitor price action on short time frames during periods of low volatility.
- Use an oscillator set at a high sensitivity to identify overbought or oversold levels.
- Plot short or long positions based on price fluctuations.
- Analyze charts on shorter time periods for better insights.

3. Gap fill:
- Mark price gaps from the previous trading day session.
- Plan for common gap stra

Final Answer: 
Page not found | AskTraders.com
Skip to content
 en
Japanese
Español
Melayu
Italiano
العربية
Deutsch
English
Shares
London Listed
Nasdaq
NYSE
Bursa Malaysia
Top Picks
Markets
FTSE 100
FTSE 250
S&P 500
Nasdaq 100
Dow Jones (DJIA)
DAX
Learn
Forex Trading
CFD Trading
Stock Trading
Crypto Trading
Trading Strategies
Technical Analysis
Trading Guides
Broker Guides
Ethical Trading
Risk Management
Compare
Best Forex Brokers
Best CFD Brokers
Best Stock Brokers
Best Crypto Brokers
Social Trading Brokers
Robo Advisors
Broker Reviews
Best Brokers
Spread Betting Brokers
News
Team
Home
Oops! That page can’t be found.
It looks like nothing was found at this location. Maybe try one of the links below or a search?
Before you trade, AskTraders.
Start Trading
Risk warning
Trading financial products carries a high risk to your capital, particularly when engaging in leveraged transactions such as CFDs. It is important to note that between 74-89% of retail investors lose money when trading CF

Final Answer: 

Risk Analysis Report for AAPL Trading Strategies:

After evaluating the proposed trading strategies for AAPL, there are several potential risks that need to be considered along with mitigation strategies:

1. Two-day high/low trading:
- Risk: The strategy relies heavily on historical data and may not account for sudden market shifts.
- Mitigation: Monitor real-time market conditions and adjust strategy accordingly. Implement strict stop-loss orders to limit potential losses.

2. Low volatility ranges:
- Risk: Overreliance on indicators may lead to false signals and erroneous trades.
- Mitigation: Combine indicator signals with price action analysis to confirm trade entries. Use tight risk management to protect capital.

3. Gap fill:
- Risk: Price gaps may not always fill as expected, leading to losses.
- Mitigation: Set clear criteria for gap trades and wait for confirmation before entering a position. Use trailing stops to lock in profits and limit potential losses.

O

- Display the final result as Markdown.

In [18]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL Trading Strategies:

After evaluating the proposed trading strategies for AAPL, there are several potential risks that need to be considered along with mitigation strategies:

1. Two-day high/low trading:
- Risk: The strategy relies heavily on historical data and may not account for sudden market shifts.
- Mitigation: Monitor real-time market conditions and adjust strategy accordingly. Implement strict stop-loss orders to limit potential losses.

2. Low volatility ranges:
- Risk: Overreliance on indicators may lead to false signals and erroneous trades.
- Mitigation: Combine indicator signals with price action analysis to confirm trade entries. Use tight risk management to protect capital.

3. Gap fill:
- Risk: Price gaps may not always fill as expected, leading to losses.
- Mitigation: Set clear criteria for gap trades and wait for confirmation before entering a position. Use trailing stops to lock in profits and limit potential losses.

Overall, it is essential to carefully monitor market conditions, adapt to changing trends, and implement strict risk management practices to mitigate potential risks associated with the proposed trading strategies for AAPL. Experimentation and testing of these strategies are crucial before applying them to real-time trades to optimize trading effectiveness.